In [1]:
import sys
sys.path.insert(0, '../utils')

import os
import cv2
import numpy as np
from typing import Dict, Tuple
from skimage import io
from dataGenerator import *

%reload_ext autoreload
%autoreload 2

In [2]:
DATA_BANK_DIRECTORY

'/home/mark/DataBank/probav_data/'

In [5]:
for _ in range(50):
    # Load dataset
    imgAllSets = generateImageSet(isTrainData=True, NIR=True)

    # Remove outliers
    imgAllSets = removeImageWithOutlierPixels(imageSet=imgAllSets, threshold=60000, isTrainData=True)

    # Upscale
    imgAllSetsUpScaled = upsampleImages(imageSets=imgAllSets, scale=3)  # 128x128 -> 384x384

    # Convert dataset to numpy array
    output = imageSetToNumpyArrayHelper(imgAllSetsUpScaled, imgAllSets, isGrayScale=True, isNHWC=False)

    # Get shifts and trim dataset based on shift threshold
    # Read more about this shifts happening in LR Images here
    # https://kelvins.esa.int/proba-v-super-resolution/data/
    saveArrays(output, DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT, 'NIR')


    correctShiftsFromSavedArrays(os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT,
                                                  'numpyArrays', 'NIR'),
                                 os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT,
                                                  'correctedShifts', 'NIR'),
                                 output['names'], upsampleScale=1)

    srcFolder = os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT, 'correctedShifts', 'NIR')
    dstFolder = os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT, 'patches', 'upsample', 'NIR')

    out = generatePatchDatasetFromSavedFile(srcFolder, dstFolder, output['names'], useUpsample=True,
                                          patchSize=96, thresholdPatchesPerImgSet=20, thresholdClarityLR=0.7,
                                          thresholdClarityHR=0.85)

    dstFolder = os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT, 'patches', 'normal', 'NIR')

    out = generatePatchDatasetFromSavedFile(srcFolder, dstFolder, output['names'], useUpsample=False,
                                          patchSize=32, thresholdPatchesPerImgSet=20, thresholdClarityLR=0.7,
                                          thresholdClarityHR=0.85)

[ INFO ] Correcting shifts in imagesets        : 100%|██████████| 562/562 [58:14<00:00,  6.22s/it] 
[ INFO ] Finding patches                       :   7%|▋         | 37/562 [02:59<37:32,  4.29s/it]

KeyboardInterrupt: 

In [6]:
output['names']

['imgset0594',
 'imgset0595',
 'imgset0596',
 'imgset0597',
 'imgset0598',
 'imgset0599',
 'imgset0600',
 'imgset0601',
 'imgset0602',
 'imgset0603',
 'imgset0604',
 'imgset0605',
 'imgset0606',
 'imgset0607',
 'imgset0608',
 'imgset0609',
 'imgset0610',
 'imgset0611',
 'imgset0612',
 'imgset0613',
 'imgset0614',
 'imgset0615',
 'imgset0616',
 'imgset0617',
 'imgset0618',
 'imgset0619',
 'imgset0620',
 'imgset0621',
 'imgset0622',
 'imgset0623',
 'imgset0624',
 'imgset0625',
 'imgset0626',
 'imgset0627',
 'imgset0628',
 'imgset0629',
 'imgset0630',
 'imgset0631',
 'imgset0632',
 'imgset0633',
 'imgset0634',
 'imgset0635',
 'imgset0636',
 'imgset0637',
 'imgset0638',
 'imgset0639',
 'imgset0640',
 'imgset0641',
 'imgset0642',
 'imgset0643',
 'imgset0644',
 'imgset0645',
 'imgset0646',
 'imgset0647',
 'imgset0648',
 'imgset0649',
 'imgset0650',
 'imgset0651',
 'imgset0652',
 'imgset0653',
 'imgset0654',
 'imgset0655',
 'imgset0656',
 'imgset0657',
 'imgset0658',
 'imgset0659',
 'imgset06

In [ ]:
    dstFolder = os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT, 'patches', 'normal', 'NIR')

    out = generatePatchDatasetFromSavedFile(srcFolder, dstFolder, output['names'], useUpsample=False,
                                          patchSize=32, thresholdPatchesPerImgSet=20, thresholdClarityLR=0.7,
                                          thresholdClarityHR=0.85)


[ INFO ] Finding patches                       :   0%|          | 0/562 [00:00<?, ?it/s]
[ INFO ] Finding patches                       :   0%|          | 1/562 [00:05<47:05,  5.04s/it]
[ INFO ] Finding patches                       :   0%|          | 2/562 [00:09<44:20,  4.75s/it]
[ INFO ] Finding patches                       :   1%|          | 3/562 [00:14<45:00,  4.83s/it]
[ INFO ] Finding patches                       :   1%|          | 4/562 [00:18<44:37,  4.80s/it]
[ INFO ] Finding patches                       :   1%|          | 5/562 [00:24<45:57,  4.95s/it]
[ INFO ] Finding patches                       :   1%|          | 6/562 [00:28<44:10,  4.77s/it]
[ INFO ] Finding patches                       :   1%|          | 7/562 [00:33<45:17,  4.90s/it]
[ INFO ] Finding patches                       :   1%|▏         | 8/562 [00:39<46:53,  5.08s/it]
[ INFO ] Finding patches                       :   2%|▏         | 9/562 [00:44<46:09,  5.01s/it]
[ INFO ] Finding patches             

[ INFO ] Finding patches                       :  28%|██▊       | 160/562 [13:28<34:04,  5.08s/it]


[ INFO ] Finding patches                       :  43%|████▎     | 240/562 [20:13<25:41,  4.79s/it]


[ INFO ] Finding patches                       :  57%|█████▋    | 320/562 [27:06<21:09,  5.25s/it]


[ INFO ] Finding patches                       :  59%|█████▊    | 330/562 [27:56<19:16,  4.98s/it]